In [69]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import math
import datetime
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [70]:
df = pd.read_csv('https://raw.githubusercontent.com/xxiang27/sikkaAIHackathon/main/data_file.csv')
df.head()

,id,month,year,visits,no_of_appts,production
0,34,3,2014,1,960,76.55
1,34,4,2014,488,1089,106333.80
2,34,5,2014,831,993,192230.40
3,34,6,2014,753,884,163515.12
4,34,7,2014,721,797,160185.90


In [71]:
df['day'] = 1
dates = pd.to_datetime(df[['month', 'year','day']].copy())
dates.head()

0   2014-03-01
1   2014-04-01
2   2014-05-01
3   2014-06-01
4   2014-07-01
dtype: datetime64[ns]

In [72]:
df_final = pd.concat([dates, df],axis=1)
df_final = df_final.drop(columns = ['month', 'year', 'day'])
df_final.columns.values[0] = 'date'
df_final.set_index('date', inplace = True)
df_final.index = pd.DatetimeIndex(df_final.index).to_period('M')


In [73]:
df_final.head()

,id,visits,no_of_appts,production
date,,,,
2014-03,34,1,960,76.55
2014-04,34,488,1089,106333.80
2014-05,34,831,993,192230.40
2014-06,34,753,884,163515.12
2014-07,34,721,797,160185.90


In [95]:
# separate dataframe into separate dataframes for each practice 

import math 
import copy

def separate(df):
  '''takes a dataframe return a list of dataframes mapped to id'''
  df_dict = {}
  grouped = df.groupby(df.id)
  for i in range(1, 285):
    df_dict[i] = grouped.get_group(i)
  return df_dict

def train_test_split(df_dict, add_test):
  '''takes a list of dataframes and returns the time series train and test split based on the percent'''
  test_set = {}
  for id, df in df_dict.items():
    df_add = df.tail(add_test).copy()
    test_set[id] = df_add
    df.drop(df.tail(add_test).index, inplace = True)
  return df_dict, test_set

def make_predictions(df_train):
  make_predictions = {}
  for id,df in df_train.items():
    X = df['production']
    #print(X)
   # X.set_index('date', inplace=True)
   # id = df.iloc[0, 1]
    #X.rename(columns={"date": "ds", "production": "y"}, inplace = True)
    my_order = (1, 1, 1)
    my_seasonal_order = (1, 1, 1, 12)
    
    model = sm.tsa.statespace.SARIMAX(X,order = my_order, seasonal_order = my_seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
    start_params = np.r_[[0] * (model.k_params - 1), 1]
    model_fit = model.fit(start_params = start_params)
    yhat = model_fit.predict(start = len(df), end = len(df)+3)
    make_predictions[id] = yhat
  return make_predictions 
    
def MAPE(y_true, y_pred):
	return np.mean(np.abs((y_true - y_pred)/y_true))*100

# need to make both of them dictionaries 
def score(predictions, actual):
  scores = []
  cate = [0,0,0,0]
  for i in range(1, 285):
    score = MAPE(actual[i]['production'].to_numpy(), predictions[i].to_numpy())
    scores.append(score)
    print(score)
    if score <= 5:
      cate[0] += 1
    elif score > 5 and score <=10:
      cate[1] += 1
    elif score >10 and score <= 15:
      cate[2] += 1
    else:
      cate[3] += 1
  total_score = (cate[0]/284 * 2) + (cate[1]/284 * 1) + (cate[2]/284 * -1) + (cate[3]/284 * -2)
  return scores, total_score 


def dentistry_season(ds):
  date = pd.to_datetime(ds)
  # high traffic
  if date.month in [11,12,1,2]:
    return 3 
  # med traffic
  if date.month in [8, 10,7, 4]:
    return 2
  # low traffic
  if date.month in [3,5,6,9]:
    return 1





In [75]:
df_dict = separate(df_final)
train, test = train_test_split(df_dict, 4)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [76]:
%%capture
preds = make_predictions(train)

In [96]:
score, total = score(preds, test)

7.099860146331712
87.67476374629794
2.104884154172347
47.03476926469957
14.273665659757818
22.80731806527877
47.33781712671022
9.592466875806581
21.350879742238718
11.67535202266086
34.158814309979135
11.399666967294209
23.247863850654486
30.831431321648267
39.40226636357519
9.355635396869037
32.565118448525304
6.923732941270836
33.37373602460976
21.79670196814567
16.227171793071953
14.919556341572404
8.344170518293367
17.210003676638962
172.13832025947644
20.591550738916204
14.143040462043968
17.601392351936656
73.323637381003
13.762645792416672
15.91844059007152
10.607061065796767
10.675724318309335
12.318496031869183
14.696731110267319
24.046243323256654
32.28887044526165
24.055767009005322
8.748381253454756
16.229952787187482
33.26528065910097
21.42834261794925
19.26505359786913
20.704646094453192
46.32812200421525
22.394226479483955
7.191797926077998
31.673562533828953
52.90067536288622
18.003527125995245
33.70337042265727
7.331540929079513
25.924337495531102
11.954871862471295
17

In [94]:
total

-1.3556338028169015

In [82]:
preds[3]

2020-09    282358.934159
2020-10    288889.652623
2020-11    300314.131213
2020-12    236852.560035
Freq: M, dtype: float64

8.253632029007123
72.56362114672078
2.104884154172347
38.194361601437585
nan
22.80731806527877
47.33781712671022
9.592466875806581
nan
11.67535202266086
nan
nan
nan
30.130780285041592
38.66962707702065
nan
nan
6.923732941270836
nan
21.79670196814567
9.260299523504575
nan
8.344170518293367
17.210003676638962
27.37418511406539
20.591550738916204
14.143040462043968
9.558775861761735
7.0664156932673485
18.784288842079384
13.882637150343552
nan
nan
12.318496031869183
14.696731110267319
14.941068635427367
19.42062168724282
nan
nan
28.952725176591144
nan
16.209408626456057
12.724134767726813
nan
36.26885219317077
22.394226479483955
nan
23.954812422032763
52.90067536288622
nan
33.70337042265727
7.331540929079513
25.924337495531102
8.657702898038293
nan
29.531777692718393
257.9094198933862
16.663055103977207
2.932054712419845
17.123275664415644
nan
nan
21.410527013845407
nan
4.651979058771152
nan
15.345624408434233
nan
40.330829998639075
16.210093788484016
58.184193828196605
nan
10.095581237838

In [90]:
test[4]

,id,visits,no_of_appts,production
date,,,,
2020-09,4,152,8,45013.02
2020-10,4,163,1,62401.47
2020-11,4,175,5,74851.50
2020-12,4,178,8,82803.24


In [91]:
preds[4]

2020-07    30200.083266
2020-08    35914.710475
2020-09    17562.369281
2020-10    52788.572340
Freq: M, dtype: float64

In [97]:
[1,2,3,4]*5


[1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]

In [98]:
np.array([1,2,3,4]) + [1,2,3,7]

array([ 2,  4,  6, 11])